# Solving CartPole using Q Learning Tabular Method

In [28]:
import gym 
import numpy as np 
import math 
import random
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv
from pymgrid import MicrogridGenerator 

In [29]:
microgrid_ = MicrogridGenerator.MicrogridGenerator(nb_microgrid=1)
microgrid_.generate_microgrid()

,load,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,battery_soc_0,battery_power_charge,battery_power_discharge,battery_capacity,battery_efficiency,battery_soc_min,battery_soc_max,battery_cost_cycle,grid_weak,grid_power_import,grid_power_export
0,960,10,1,0.1,969.6,0.2,430,430,1718,0.9,0.2,1,0.02,0,1920,1920


In [30]:
mg = microgrid_.microgrids[0]
env= MicroGridEnv({'microgrid':mg })

In [31]:
result = []
observ = env.reset()
for t in range(300):
    env.render()
    action = env.action_space.sample()
    observ, reward, done, info = env.step(action)
    result.append([action, reward])

state: [0.31166667 0.         0.2        0.88888242 0.         1.
 1.         1.         0.         0.         1.        ] reward: 0 info: {}
state: [0.31250886 0.         0.2        0.88888889 0.         1.
 1.01314775 1.         0.         0.         1.        ] reward: -8.440451692118966 info: {}
state: [0.27490647 0.         0.2        0.88888889 0.         1.
 1.07797197 1.         0.         0.25881905 0.96592583] reward: -8.45583337869933 info: {}
state: [0.27486683 0.         0.2        0.88888889 0.         1.
 1.05447011 1.         0.         0.5        0.8660254 ] reward: -7.950303303346758 info: {}
state: [0.29285856 0.         0.2        0.88888889 0.         1.
 0.91365474 1.         0.         0.70710678 0.70710678] reward: -51.36848775078946 info: {}
state: [0.32367626 0.         0.42526193 0.63859785 0.20273574 1.
 0.81864712 1.         0.         0.8660254  0.5       ] reward: -13.29688346105016 info: {}
state: [0.53950529 0.         0.42526193 0.63859785 0.20273574 1

state: [ 0.75513291  0.          0.72189885  0.30900128  0.46970896  1.
  1.68462233  1.31818182  0.         -0.96592583  0.25881905] reward: -9.27439007052669 info: {}
state: [ 0.73017295  0.          0.72189885  0.30900128  0.46970896  1.
  1.68790584  1.          0.         -0.8660254   0.5       ] reward: -85.1426801137509 info: {}
state: [ 0.56490826  0.          0.72189885  0.30900128  0.46970896  1.
  1.69305867  1.          0.         -0.70710678  0.70710678] reward: -13.252126366464546 info: {}
state: [ 0.38965684  0.          0.72189885  0.30900128  0.46970896  1.
  1.67591623  1.          0.         -0.5         0.8660254 ] reward: -11.658493802393718 info: {}
state: [ 0.28609909  0.          0.72189885  0.30900128  0.46970896  1.
  1.59290549  1.          0.         -0.25881905  0.96592583] reward: -9.67666615900578 info: {}
state: [0.26192564 0.         0.94716078 0.05871024 0.6724447  1.
 1.57158096 1.         0.         0.         1.        ] reward: -13.562151700522508 

In [32]:
env.close()

In [33]:
#actions are move cart left or right 
print(env.action_space)

Discrete(5)


In [34]:
# observation space: has four values 
'''
load'
'hour'
'pv'
'battery_soc'
'capa_to_charge'
'capa_to_discharge'
'grid_status'
'grid_co2'
'grid_price_import'
'grid_price_export'
'''
print(env.observation_space)

Box(11,)


In [35]:
print(env.observation_space.low)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


In [36]:
print(env.observation_space.high)

[inf inf inf inf inf inf inf inf inf inf inf]


In [37]:
mg0 = microgrid_.microgrids[0]

######  Discretize the state space so we can apply Q learning to a bounded space

In [38]:
# load limits
mg0.forecast_load().min(), mg0.forecast_load().max()

(250.3161665960639, 810.0345883872147)

In [39]:
# pv limits
mg0.forecast_pv().min(), mg0.forecast_pv().max()

(0.0, 420.8553788587465)

In [40]:
# battery.soc limits
mg0.battery.soc_min, mg0.battery.soc_max

(0.2, 1)

In [41]:
num_buckets = [15,1,15,15,1,1,1,1,1,1,1]
num_actions = env.action_space.n
state_bounds = list(zip(env.observation_space.low, env.observation_space.high))

In [42]:
# amend the state bound ranges for velocity and pole angle 
state_bounds[0] = [mg0.forecast_load().min(), mg0.forecast_load().max()]
state_bounds[2] = [mg0.forecast_pv().min(), mg0.forecast_pv().max()]
state_bounds[3] = [mg0.battery.soc_min, mg0.battery.soc_max]
state_bounds

[[250.3161665960639, 810.0345883872147],
 (-1.0, inf),
 [0.0, 420.8553788587465],
 [0.2, 1],
 (-1.0, inf),
 (-1.0, inf),
 (-1.0, inf),
 (-1.0, inf),
 (-1.0, inf),
 (-1.0, inf),
 (-1.0, inf)]

In [43]:
q_table = np.zeros(num_buckets +[num_actions])
print(q_table.shape)
q_table

(15, 1, 15, 15, 1, 1, 1, 1, 1, 1, 1, 5)


array([[[[[[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          ...,







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]]],








         [[[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          ...,







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]]],








         [[[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          ...,







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0., 0., 0.]]]]]]]],







          [[[[[[[[0., 0., 0.,

In [44]:
explore_rate_min = 0.01
learning_rate_min = 0.1

In [45]:
def get_explore_rate(t):
    return max(explore_rate_min, min(1, 1.0-math.log10((t+1)/25)))

In [46]:
def get_learning_rate(t):
    return max(learning_rate_min,min(1, 1.0-math.log10((t+1)/25)))

In [47]:
def select_action(state, explore_rate):
    if random.random() < explore_rate:
        action = env.action_space.sample()
    else: 
        action = np.argmax(q_table[state])
    return action

In [48]:
def state_to_bucket(state):
    bucket_indices =[]
    for i in range(len(state)):
        if state[i]<=state_bounds[i][0]:
            bucket_index = 0
        elif  state[i]>=state_bounds[i][1]:
            bucket_index = num_buckets[i]-1
        else:
            bound_width = state_bounds[i][1]-state_bounds[i][0]
            offset = (num_buckets[i]-1)* state_bounds[i][0]/bound_width
            scaling = (num_buckets[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i]-offset))

        bucket_indices.append(bucket_index)

    return tuple(bucket_indices)

In [49]:
def simulate():
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    
    discount_factor = 0.99
    num_streaks = 0
    for episode in range(1000):
        observ = env.reset()
        state_0 = state_to_bucket(observ)
        done = False
        total_reward = 0
        i=0
        while not done:
            action = select_action(state_0, explore_rate)
            observ, reward, done, info = env.step(action)
            state = state_to_bucket(observ)
            best_q = np.amax(q_table[state])            
            q_table[state_0 + (action,)] += \
            learning_rate* (reward + discount_factor*(best_q) - q_table[state_0+ (action, ) ])
            state_0 = state
            i +=1
            total_reward += reward
        print(i, total_reward)
#         print("\nEpisode = %d" % episode)
#         print('t=%d' %t)
#         print('Action: %d' %action)
#         print('State:', state)
#         print('Reward:', reward)
#         print('Best Q:', best_q)
        print('Explore rate:', explore_rate)
        print('Learning rate:', learning_rate)
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)


In [50]:
simulate()

5847 -182813.92817804238
Explore rate: 1
Learning rate: 1
5847 -186234.39808217008
Explore rate: 1
Learning rate: 1
5847 -183622.3062038648
Explore rate: 1
Learning rate: 1
5847 -183245.12670726902
Explore rate: 1
Learning rate: 1
5847 -181343.07360169388
Explore rate: 1
Learning rate: 1
5847 -186824.28679331264
Explore rate: 1
Learning rate: 1
5847 -184258.77402341284
Explore rate: 1
Learning rate: 1
5847 -180303.598340711
Explore rate: 1
Learning rate: 1
5847 -182954.02937766633
Explore rate: 1
Learning rate: 1
5847 -182598.59450073796
Explore rate: 1
Learning rate: 1
5847 -182200.46236532656
Explore rate: 1
Learning rate: 1
5847 -188306.92790865642
Explore rate: 1
Learning rate: 1
5847 -180740.74891552544
Explore rate: 1
Learning rate: 1
5847 -182480.20288756385
Explore rate: 1
Learning rate: 1
5847 -180801.97647504954
Explore rate: 1
Learning rate: 1
5847 -184051.57692041123
Explore rate: 1
Learning rate: 1
5847 -183436.3676414655
Explore rate: 1
Learning rate: 1
5847 -182037.40946

5847 -105165.91849571037
Explore rate: 0.4023048140744877
Learning rate: 0.4023048140744877
5847 -106483.0788077504
Explore rate: 0.3979400086720376
Learning rate: 0.3979400086720376
5847 -106688.20139422543
Explore rate: 0.3936186348893951
Learning rate: 0.3936186348893951
5847 -103297.60298536812
Explore rate: 0.3893398369101201
Learning rate: 0.3893398369101201
5847 -105462.7672324896
Explore rate: 0.38510278396686537
Learning rate: 0.38510278396686537
5847 -100525.9877478219
Explore rate: 0.38090666937325723
Learning rate: 0.38090666937325723
5847 -105696.67182071655
Explore rate: 0.37675070960209955
Learning rate: 0.37675070960209955
5847 -104107.86801831894
Explore rate: 0.3726341434072673
Learning rate: 0.3726341434072673
5847 -104475.29241130961
Explore rate: 0.368556230986828
Learning rate: 0.368556230986828
5847 -100437.4755223385
Explore rate: 0.36451625318508785
Learning rate: 0.36451625318508785
5847 -101756.7333750756
Explore rate: 0.3605135107314139
Learning rate: 0.3605

5847 -70983.93082452328
Explore rate: 0.11918640771920863
Learning rate: 0.11918640771920863
5847 -69565.20925873499
Explore rate: 0.11690664142431006
Learning rate: 0.11690664142431006
5847 -68288.04653557418
Explore rate: 0.11463877996848804
Learning rate: 0.11463877996848804
5847 -69799.23669836334
Explore rate: 0.11238269966426384
Learning rate: 0.11238269966426384
5847 -69176.17343443821
Explore rate: 0.11013827874181159
Learning rate: 0.11013827874181159
5847 -69545.65682117644
Explore rate: 0.10790539730951965
Learning rate: 0.10790539730951965
5847 -69159.03760936997
Explore rate: 0.10568393731556158
Learning rate: 0.10568393731556158
5847 -69511.45958737844
Explore rate: 0.1034737825104447
Learning rate: 0.1034737825104447
5847 -68286.08319448026
Explore rate: 0.10127481841050645
Learning rate: 0.10127481841050645
5847 -67211.2175072702
Explore rate: 0.099086932262331
Learning rate: 0.1
5847 -69286.28850279187
Explore rate: 0.09691001300805646
Learning rate: 0.1
5847 -66942.87

5847 -55415.05213743125
Explore rate: 0.01
Learning rate: 0.1
5847 -55685.04960188003
Explore rate: 0.01
Learning rate: 0.1
5847 -55668.979423715646
Explore rate: 0.01
Learning rate: 0.1
5847 -55866.12604451528
Explore rate: 0.01
Learning rate: 0.1
5847 -55133.414476472244
Explore rate: 0.01
Learning rate: 0.1
5847 -55706.13259983714
Explore rate: 0.01
Learning rate: 0.1
5847 -55434.56116835357
Explore rate: 0.01
Learning rate: 0.1
5847 -55092.28636852924
Explore rate: 0.01
Learning rate: 0.1
5847 -55333.52558265681
Explore rate: 0.01
Learning rate: 0.1
5847 -55141.71450382776
Explore rate: 0.01
Learning rate: 0.1
5847 -55068.46058777616
Explore rate: 0.01
Learning rate: 0.1
5847 -54967.134484167436
Explore rate: 0.01
Learning rate: 0.1
5847 -55486.358615207086
Explore rate: 0.01
Learning rate: 0.1
5847 -55921.72968159132
Explore rate: 0.01
Learning rate: 0.1
5847 -56055.16836337515
Explore rate: 0.01
Learning rate: 0.1
5847 -55385.980493754985
Explore rate: 0.01
Learning rate: 0.1
584

5847 -55431.85261426505
Explore rate: 0.01
Learning rate: 0.1
5847 -55482.297541668486
Explore rate: 0.01
Learning rate: 0.1
5847 -55257.683338666604
Explore rate: 0.01
Learning rate: 0.1
5847 -55252.10951081028
Explore rate: 0.01
Learning rate: 0.1
5847 -55776.405845666755
Explore rate: 0.01
Learning rate: 0.1
5847 -55297.87156369059
Explore rate: 0.01
Learning rate: 0.1
5847 -55201.618131498755
Explore rate: 0.01
Learning rate: 0.1
5847 -55233.99896110796
Explore rate: 0.01
Learning rate: 0.1
5847 -55628.50323551394
Explore rate: 0.01
Learning rate: 0.1
5847 -55586.40903474137
Explore rate: 0.01
Learning rate: 0.1
5847 -55436.52314765718
Explore rate: 0.01
Learning rate: 0.1
5847 -55631.57313224008
Explore rate: 0.01
Learning rate: 0.1
5847 -55495.41316135639
Explore rate: 0.01
Learning rate: 0.1
5847 -55406.21622311594
Explore rate: 0.01
Learning rate: 0.1
5847 -55123.04503739848
Explore rate: 0.01
Learning rate: 0.1
5847 -55341.806908108694
Explore rate: 0.01
Learning rate: 0.1
584

5847 -55171.334411767544
Explore rate: 0.01
Learning rate: 0.1
5847 -55239.61554148213
Explore rate: 0.01
Learning rate: 0.1
5847 -55832.52822891761
Explore rate: 0.01
Learning rate: 0.1
5847 -55455.08644485846
Explore rate: 0.01
Learning rate: 0.1
5847 -56162.732688344775
Explore rate: 0.01
Learning rate: 0.1
5847 -55472.435963383796
Explore rate: 0.01
Learning rate: 0.1
5847 -55651.08180941652
Explore rate: 0.01
Learning rate: 0.1
5847 -55463.04746377256
Explore rate: 0.01
Learning rate: 0.1
5847 -55458.405278609156
Explore rate: 0.01
Learning rate: 0.1
5847 -55449.68643704081
Explore rate: 0.01
Learning rate: 0.1
5847 -55648.73764334393
Explore rate: 0.01
Learning rate: 0.1
5847 -55161.07227938569
Explore rate: 0.01
Learning rate: 0.1
5847 -55511.93326204723
Explore rate: 0.01
Learning rate: 0.1
5847 -55069.576331583754
Explore rate: 0.01
Learning rate: 0.1
5847 -55096.90824500583
Explore rate: 0.01
Learning rate: 0.1
5847 -55586.62540798068
Explore rate: 0.01
Learning rate: 0.1
584

5847 -55142.43378671342
Explore rate: 0.01
Learning rate: 0.1
5847 -55162.104137457456
Explore rate: 0.01
Learning rate: 0.1
5847 -55071.704815701676
Explore rate: 0.01
Learning rate: 0.1
5847 -55106.84831341907
Explore rate: 0.01
Learning rate: 0.1
5847 -54791.00949340474
Explore rate: 0.01
Learning rate: 0.1
5847 -55108.78295813613
Explore rate: 0.01
Learning rate: 0.1
5847 -55897.64343584968
Explore rate: 0.01
Learning rate: 0.1
5847 -55245.768987438285
Explore rate: 0.01
Learning rate: 0.1
5847 -55477.668182895715
Explore rate: 0.01
Learning rate: 0.1
5847 -55345.39324352771
Explore rate: 0.01
Learning rate: 0.1
5847 -55297.05724810076
Explore rate: 0.01
Learning rate: 0.1
5847 -55275.84357635231
Explore rate: 0.01
Learning rate: 0.1
5847 -55073.267805221825
Explore rate: 0.01
Learning rate: 0.1
5847 -55399.42294453225
Explore rate: 0.01
Learning rate: 0.1
5847 -55558.32490420987
Explore rate: 0.01
Learning rate: 0.1
5847 -55153.944886571386
Explore rate: 0.01
Learning rate: 0.1
58

5847 -54970.35048898901
Explore rate: 0.01
Learning rate: 0.1
5847 -56234.58393076207
Explore rate: 0.01
Learning rate: 0.1
5847 -55078.75453730823
Explore rate: 0.01
Learning rate: 0.1
5847 -55236.57449741777
Explore rate: 0.01
Learning rate: 0.1
5847 -55647.06353512972
Explore rate: 0.01
Learning rate: 0.1
5847 -55249.03961281374
Explore rate: 0.01
Learning rate: 0.1
5847 -55319.34796749635
Explore rate: 0.01
Learning rate: 0.1
5847 -55286.267685955936
Explore rate: 0.01
Learning rate: 0.1
5847 -55219.46236413759
Explore rate: 0.01
Learning rate: 0.1
5847 -56492.12363403836
Explore rate: 0.01
Learning rate: 0.1
5847 -55263.45338960174
Explore rate: 0.01
Learning rate: 0.1
5847 -55235.51672198405
Explore rate: 0.01
Learning rate: 0.1
5847 -55379.18754703321
Explore rate: 0.01
Learning rate: 0.1
5847 -55547.15354682257
Explore rate: 0.01
Learning rate: 0.1
5847 -55679.75994139521
Explore rate: 0.01
Learning rate: 0.1
5847 -55112.63203180662
Explore rate: 0.01
Learning rate: 0.1
5847 -5

5847 -55167.83297605265
Explore rate: 0.01
Learning rate: 0.1
5847 -55118.211644306524
Explore rate: 0.01
Learning rate: 0.1
5847 -55163.68968165614
Explore rate: 0.01
Learning rate: 0.1
5847 -55641.39154299557
Explore rate: 0.01
Learning rate: 0.1
5847 -54817.36731168348
Explore rate: 0.01
Learning rate: 0.1
5847 -55024.174032074814
Explore rate: 0.01
Learning rate: 0.1
5847 -55136.26197195718
Explore rate: 0.01
Learning rate: 0.1
5847 -55570.315227528925
Explore rate: 0.01
Learning rate: 0.1
5847 -55547.68650329165
Explore rate: 0.01
Learning rate: 0.1
5847 -55678.70716213642
Explore rate: 0.01
Learning rate: 0.1
5847 -55764.64811012081
Explore rate: 0.01
Learning rate: 0.1
5847 -55679.78124944957
Explore rate: 0.01
Learning rate: 0.1
5847 -55491.04553911362
Explore rate: 0.01
Learning rate: 0.1
5847 -55281.87449524499
Explore rate: 0.01
Learning rate: 0.1
5847 -55166.668460746914
Explore rate: 0.01
Learning rate: 0.1
5847 -55089.3665928032
Explore rate: 0.01
Learning rate: 0.1
5847 

######  State action table 

In [51]:
q_table

array([[[[[[[[[[[[ -976.65991019,  -934.52385499,
                   -942.38592963,  -984.28076416,
                   -942.28980977]]]]]]]],







          [[[[[[[[ -989.87257962,  -941.98790413,
                   -944.52345608,  -973.00304374,
                   -944.7625593 ]]]]]]]],







          [[[[[[[[ -986.03743956,  -935.26026806,
                   -945.74199413,  -991.7137901 ,
                   -943.10105464]]]]]]]],







          ...,







          [[[[[[[[-1013.85159223,  -962.39510411,
                   -963.16841905, -1013.28503777,
                   -954.65675197]]]]]]]],







          [[[[[[[[    0.        ,     0.        ,
                      0.        ,     0.        ,
                      0.        ]]]]]]]],







          [[[[[[[[    0.        ,     0.        ,
                      0.        ,     0.        ,
                      0.        ]]]]]]]]],








         [[[[[[[[[    0.        ,     0.        ,
                      0.        

In [25]:
q_table

array([[[[[[[[[[[[ -977.27410087,  -938.34668301,
                   -941.78343137,  -975.39278143,
                   -947.58512623]]]]]]]],







          [[[[[[[[ -991.69305357,  -942.06249393,
                   -949.67919933,  -991.24979579,
                   -951.22954521]]]]]]]],







          [[[[[[[[ -999.72703029,  -960.19980483,
                   -958.51445498,  -999.62118278,
                   -952.23616095]]]]]]]],







          [[[[[[[[-1018.17996205,  -966.00504884,
                   -963.90874811, -1012.71137615,
                   -955.89837984]]]]]]]],







          [[[[[[[[    0.        ,     0.        ,
                      0.        ,     0.        ,
                      0.        ]]]]]]]]],








         [[[[[[[[[    0.        ,     0.        ,
                      0.        ,     0.        ,
                      0.        ]]]]]]]],







          [[[[[[[[    0.        ,     0.        ,
                      0.        ,     0.        ,
    

In [ ]:
microgrid_.microgrids[0].benchmarks.run_mpc_benchmark()
microgrid_.microgrids[0].benchmarks.describe_benchmarks()